Course: ISTE 782
Student: Ian Effendi (iae2784@rit.edu)

# Assignment 2: Data Scavenger Hunt

> The purpose of this assignment is to give you the opportunity to manipulate, display, and compute features from raw data. Over the course of this assignment, you will be guided through a process of manually finding interesting phenomena in a data set. There is no specific process that will work for all data sets, but this specific process should give you a feel for how you would initially analyze your own data set that you’ll find for your course project.

## Table of Contents

- [Part 1: Loading Data in Python](#part1)
- [Part 2: Data Manipulation and Visualization](#part2)
- [Part 3: Creating, Analyzing, and Selecting Features](#part3)
- [Part 4: Analyzing Selected Features](#part4)

## <a name="part1"> Part 1: Loading Data in Python

In [1]:
# Import dependencies.
import numpy as np
import matplotlib.pyplot as plt

# Style the graphs.
plt.style.use('fivethirtyeight')

In [2]:
# prepare conversion constants
conversion_factor = 1000

In [3]:
# make empty data and time Lists
LA_list = []
RV_list = []
RA_list = []
TL_list = []
TR_list = []

In [4]:
# Read in the waveforms dataset.
def read_waveforms(LA, RV, RA) :
    infile = open('waveforms.csv', 'r')

    line = infile.readline()
    wf = 0 # which waveform are we trying to read (0 = LA, 1 = RV, 2 = RA)

    while line :
        line = line.strip()
        data = line.split(',')

        for i in range(0, len(data)) : 
            data[i] = float(data[i])

        if(wf == 0) :
            LA.append(data)
        elif(wf == 1) :
            RV.append(data)
        elif(wf == 2) :
            RA.append(data)
        
        wf = (wf + 1) % 3
        line = infile.readline()

    infile.close()

We modify the `read_times` function to apply a conversion factor of `x1000` for both the linear and rotational times.

In [5]:
# Read times dataset.
def read_times(TL, TR) :
    infile = open('times.csv', 'r')
    line = infile.readline()
    data = line.strip().split(',')

    for i in range(0, len(data)) : 
        data[i] = float(data[i]) * conversion_factor

    TL.append(data)

    line = infile.readline()
    data = line.strip().split(',')

    for i in range(0, len(data)) : 
        data[i] = float(data[i]) * conversion_factor

    TR.append(data)
    infile.close()

In [6]:
# Read in the datasets.
read_waveforms(LA_list, RV_list, RA_list)
read_times(TL_list, TR_list)

# convert all data and time lists to numpy arrays for plotting
LA = np.array(LA_list)
RV = np.array(RV_list)
RA = np.array(RA_list)
TL = np.array(TL_list[0])
TR = np.array(TR_list[0])

## <a name="part2"> Part 2: Data Manipulation and Visualization

Let's plot the linear acceleration waveform to see what it looks like.

In [7]:
# Plot instance helper function.
def plot_instance(data, time, title, xlab, ylab, outfile):
    plt.plot(time, data)
    plt.title(title)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.xticks(np.arange(0, 55, step=5))
    plt.savefig(outfile)
    plt.close()    

In [8]:
# Create the plot for instance 7 of the LA dataset.
plot_instance(
    LA[6,:], TL,
    "Linear Acceleration of Instance 7",
    "Time (ms)",
    "Lin. Accel. (g)",
    "Instance 7.png")

Using a `for` loop and the `subplot` command, we can create an image file for every data instance that contains all three waveforms (for that instance) in a single image.

In [9]:
# Fill the stub function for plotting waveforms.
def plot_waveforms(LA, RV, RA, TL, TR) :
        
    # Function reference for plotting an instance
    # of data (an array) time series.
    def plot_instance(data, time, ylabel, yticks):
        plt.plot(time, data)
        plt.ylabel(ylabel)
        plt.xticks(np.arange(0, 55, step=5))
        plt.yticks(yticks) 
    
    # Get x- or y-axis ticks.
    def get_ticks(min_value, max_value):
        
        # Get the value range.
        extents = abs(max_value - min_value)
        
        # If value range is zero, set steps to 1.
        if extents == 0:
            return np.arange(min_value - 1, max_value + 2, step=1)
        
        if extents > 25 and extents <= 40:
            step = 5
        else:
            step = extents / 10
        
        # If step is zero, default to 1.
        if step == 0:
            step = 1
        
        return np.arange(0, max_value + 1, step=step)
    
    n_samples = len(LA)
    for i in range(0, n_samples):
        
        # Create the outfile name for this chart.
        outfile = f"figures/[{i:02d}]_waveform_instance_{i+1}.png"
        
        # Set the figure size.
        fig = plt.figure(figsize=(15, 12))
        
        # Set the title for this batch of charts.
        fig.suptitle(f"Waveforms for Instance {i+1}", fontsize=16)
                
        # LA
        plt.subplot(311)
        plot_instance(LA[i,:], TL, "Lin. Accel. (g)", get_ticks(0, np.max(LA[i,:])))
        
        # RV
        plt.subplot(312)
        plot_instance(RV[i,:], TR, "Rot. Vel. (rad/sec)", get_ticks(0, np.max(RV[i,:])))
        
        # RA
        plt.subplot(313)
        plot_instance(RA[i,:], TR, "Rot. Accel. (rad/sec^2)",  get_ticks(0, np.max(RA[i,:])))
        
        # Add x-axis label.
        plt.xlabel("Time (ms)")
        
        # Close this chart after saving it.
        plt.savefig(outfile)
        plt.close()    

In [10]:
# Execute drawing (and time process) for all charts.
%time plot_waveforms(LA, RV, RA, TL, TR)

CPU times: user 12.5 s, sys: 408 ms, total: 12.9 s
Wall time: 12.9 s


> Do any of the waveforms standout over the others? Compare the respective individual waveforms from each data instance as you go through the images one at a time. Is there a way to improve the visualizations to make the waveform comparisons more meaningful?

- Additional guidelines, rules, and reference lines to identify or highlight interesting points.
    - Addition of guidelines showing aggregated summary statistics per instance.
    - Addition of guidelines showing aggregated summary statistics across all instances.
- Shaded overlap regions of all instances showing the "hottest" spots.
    - Each line could have a very small alpha value (eg. 1/65).
    - If we draw all lines overlapping, the boldest visual area will correspond with the most frequent peaking times across all instances.

## <a name="part3"> Part 3: Creating, Analyzing, and Selecting Features

> Next, write code that will iterate across the three respective waveform matrices (LA, RV, and RA) and compute the min, max, and mean of each individual waveform. The easiest way to do this is to append the feature values to lists and then convert the lists to numpy arrays. Use the following names for your numpy arrays.

In [11]:
# Generate summary feature using aggregate functions.
def aggregate(data, func):
    n_rows = np.shape(data)[0] # Calculate the rows.
    return np.array([func(data[x,:]) for x in range(n_rows)])

In [12]:
def summarize_instance(data):
    return {
        "minimum": aggregate(data, np.min),
        "average": aggregate(data, np.mean),
        "peak": aggregate(data, np.max)
    };

In [13]:
def summarize_waveforms(LA, RV, RA):
    return {
       "LA": LA,
       "MLA": summarize_instance(LA)['minimum'], # Minimum linear acceleration.
       "ALA": summarize_instance(LA)['average'], # Average linear acceleration.
       "PLA": summarize_instance(LA)['peak'],    # Peak linear acceleration.
        
       "RV": RV,
       "MRV": summarize_instance(RV)['minimum'], # Minimum linear acceleration.
       "ARV": summarize_instance(RV)['average'], # Average linear acceleration.
       "PRV": summarize_instance(RV)['peak'],    # Peak linear acceleration.
        
       "RA": RA,
       "MRA": summarize_instance(RA)['minimum'], # Minimum linear acceleration.
       "ARA": summarize_instance(RA)['average'], # Average linear acceleration.
       "PRA": summarize_instance(RA)['peak'],    # Peak linear acceleration.
    }

In [14]:
# Get waveform summaries.
waveform_summary = summarize_waveforms(LA, RV, RA)

In [15]:
# Summary function for repeatedly printing aggregate info about a feature column.
def summarise(df, feature, view=0):
    if not view and feature in df.keys():
        df_subset = df
    elif view and feature in view and feature in df.keys():
        df_subset = { key: value for key, value in df.items() if key in view }
    else:
        return
    field = df_subset[feature]
    print(f'{feature}: min = {np.min(field)}, max = {np.max(field)}, avg = {np.mean(field)}')

In [16]:
# Summarise the LA features.
for feature in waveform_summary.keys():
    summarise(waveform_summary, feature, [ 'LA', 'MLA', 'ALA', 'PLA' ] )

LA: min = 0.0, max = 151.7148, avg = 10.964327359823077
MLA: min = 0.0, max = 0.0, avg = 0.0
ALA: min = 4.39519404, max = 22.2569035, avg = 10.964327359823077
PLA: min = 16.8252, max = 151.7148, avg = 50.71578769230769


In [17]:
# Summarise the RV features.
for feature in waveform_summary.keys():
    summarise(waveform_summary, feature, [ 'RV', 'MRV', 'ARV', 'PRV' ] )

RV: min = 0.0, max = 55.3868, avg = 10.961333207300838
MRV: min = 0.0, max = 0.0, avg = 0.0
ARV: min = 5.7067274257425735, max = 23.397538128712867, avg = 10.961333207300836
PRV: min = 18.1009, max = 55.3868, avg = 35.96904461538462


In [18]:
# Summarise the RA features.
for feature in waveform_summary.keys():
    summarise(waveform_summary, feature, [ 'RA', 'MRA', 'ARA', 'PRA' ] )

RA: min = 0.0, max = 44.5424, avg = 2.070695177913176
MRA: min = 0.0, max = 0.0, avg = 0.0
ARA: min = 1.0092340495049505, max = 4.418352356435643, avg = 2.0706951779131764
PRA: min = 10.3132, max = 44.5424, avg = 13.493876923076924


In [19]:
df_features = { key: value for key, value in waveform_summary.items() if key not in [ 'LA', 'RV', 'RA' ] }
df_features

{'MLA': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'ALA': array([ 9.86977313,  9.17452411,  8.16959799,  8.78846438,  8.25688312,
        14.15928976,  8.39712063, 15.92717737, 16.45278075,  6.73995713,
        22.2569035 , 18.36913485, 13.06193737, 13.10548129,  7.75157758,
         9.31878412,  7.24478211,  5.85639257, 18.11082306, 13.05662022,
         8.8751908 ,  9.10981387,  8.16117969, 13.30993375, 12.55893125,
        12.92154013, 11.7507689 ,  9.073431  ,  7.16341877, 16.57517157,
         7.95070516, 12.05786725,  8.47340575, 12.14093163, 13.10117613,
         7.03184212,  5.66922813,  6.24703234, 10.94524884,  7.15529613,
        10.96413075, 10.48522083,  5.45479125,  4.39519404,  6.40556837,
         9.0184685 , 19.75237338, 1

> Create line plots of each of the features and visually compare them. The plotting you do is up to you.

In [20]:
# Get the output filename.
def get_filename(tag, outdir):
    return f"{outdir}/{tag}.png"

In [21]:
# Get the title.
def get_title(feature_name):
     return {
        "MLA": "Minimum Linear Acceleration",
        "ALA": "Average Linear Acceleration",
        "PLA": "Peak Linear Acceleration",

        "MRV": "Minimum Rotational Velocity",
        "ARV": "Average Rotational Velocity",
        "PRV": "Peak Rotational Velocity",

        "MRA": "Minimum Rotational Acceleration",
        "ARA": "Average Rotational Acceleration",
        "PRA": "Peak Rotational Acceleration",
    }[feature_name] 

In [22]:
# Get the xlabel.
def get_label(feature_name):
    return {
        "MLA": "Lin. Accel. (g)",
        "ALA": "Lin. Accel. (g)",
        "PLA": "Lin. Accel. (g)",

        "MRV": "Rot. Vel. (rad/s)",
        "ARV": "Rot. Vel. (rad/s)",
        "PRV": "Rot. Vel. (rad/s)",

        "MRA": "Rot. Accel. (rad/s^2)",
        "ARA": "Rot. Accel. (rad/s^2)",
        "PRA": "Rot. Accel. (rad/s^2)",
    }[feature_name]    

In [23]:
# Get x- or y-axis ticks.
def get_ticks(min_value, max_value):

    # Get the value range.
    extents = abs(max_value - min_value)

    # If value range is zero, set steps to 1.
    if extents == 0:
        return np.arange(min_value - 1, max_value + 2, step=1)

    if extents > 25 and extents <= 40:
        step = 5
    else:
        step = extents / 10

    # If step is zero, default to 1.
    if step == 0:
        step = 1

    return np.arange(0, max_value + 1, step=step)

In [24]:
# Fill the stub function for plotting waveforms.
def plot_features(df) :
        
    # Function reference for plotting an instance
    # of data (an array) time series.
    def plot_feature(y, title, xlab, ylab, xticks, yticks, outfile):
        plt.plot(y)
        plt.title(title)
        plt.xlabel(xlab)
        plt.ylabel(ylab)
        plt.xticks(xticks)
        plt.yticks(yticks)
        plt.savefig(outfile)
        plt.close()    
                        
    # xlabel is the same.
    xlab = "Instances"
    
    # xticks is the same.
    xticks = np.arange(0, 66, step=5)
        
    # Loop through each feature.
    for feature in df.keys():
        
        # Get the data series based on the feature name.
        ds = df[feature]

        # Create the outfile name for this chart.
        f_outfile = get_filename(feature + "_", 'figures')
        
        # Get the title.
        f_title = get_title(feature)
        
        # Get the x label.
        f_ylab = get_label(feature)
    
        # Get the x ticks.
        f_yticks = get_ticks(np.min(ds), np.max(ds))
    
        # Set the figure size.
        plt.figure(figsize=(15, 12))        
        
        # Plot the feature.
        plot_feature(ds, f_title, xlab, f_ylab, xticks, f_yticks, f_outfile)

In [25]:
%time plot_features(df_features)

CPU times: user 856 ms, sys: 28.9 ms, total: 884 ms
Wall time: 881 ms


> After visualizing all nine of the features and pondering the summary statistics, three features will appear to be the most useful for differentiating between the 'different' data instances and the rest of them. Make sure that you save all of your features plots for this assignment since you will be investigating three of your selected features by visualizing them in different ways.

I selected three features:
- `PLA`: Peak Linear Acceleration
    - Visually identified outliers near instances `5`, `10`, and `20`.
- `PRA`: Peak Rotational Acceleration
    - Visually identified significant outliers near instances `5`, `10`, and `20`. May be correlated with `PLA`.
- `ARV`: Average Rotational Velocity
    - Visually identified outliers near instances `5`, `10`, `20`, `25`, and `60`.
    
At this stage of analysis, I'm not sure of the exact instances where these high values are extremely different, but these three features' visual representations suggest further investigation is necessary.

## <a name="part4"> Part 4: Analyzing Selected Features

> Using your three selected features, create three separate scatter plots (one for each combination of features).

In [26]:
# Select analysis features.
df_analysis = { key: value for key, value in waveform_summary.items() if key in [ 'PLA', 'PRA', 'ARV' ] }
df_analysis

{'PLA': array([ 31.0846,  42.3851,  39.105 ,  38.055 ,  30.3696,  54.7442,
         42.5714, 110.9405, 131.7017,  35.1772, 130.7306,  63.9629,
         52.0344,  27.035 ,  20.5185,  42.3097,  39.612 ,  31.1734,
        127.1141, 151.7148,  56.7477,  27.3868,  39.6305,  37.1073,
         62.057 ,  44.4492,  45.8111,  26.1897,  36.8324,  78.4129,
         36.7765,  31.1608,  26.8979,  35.56  ,  76.8883,  33.3552,
         35.0214,  33.9518,  70.3428,  48.5514,  36.5491,  69.637 ,
         23.076 ,  18.223 ,  21.1446,  30.9266,  75.1122,  68.9492,
         29.8736,  31.2138,  73.602 ,  16.8252,  28.7676,  43.1111,
         68.0626,  78.3577,  48.6229,  47.0987,  72.9067,  43.7065,
         52.2171,  44.8825,  59.8611,  52.5665,  35.7625]),
 'ARV': array([11.63802584,  8.74369781,  8.11205535,  7.26610713, 13.43695545,
         9.41976356,  8.96973515, 23.39753813, 20.28832089,  7.26403582,
        12.00645167, 10.38958881, 14.61630815, 21.36159765,  9.03757723,
         8.07522832,  7.520

In [27]:
# Plot scatterplots of two series.
def plot_matrix(series_x, series_y, title, xlab, ylab, xticks, yticks, outfile, figsize=(15,15), s=100, alpha=1, **kwargs):
        
    # Set figure size.
    plt.figure(figsize=figsize)
    
    # Make scatterplot.
    plt.scatter(series_x, series_y, s=s, alpha=alpha, **kwargs)
    
    # Annotate plot.    
    plt.title(title)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.xticks(xticks)
    plt.yticks(yticks)
    plt.savefig(outfile)
    plt.close()

In [28]:
# Get unique combinations of the features.
from itertools import combinations

# Get unique pairs of feature keys.
def get_unique_pairs(df):
    return [ pair for pair in combinations(df.keys(), 2) ]

# Get series_x and series_y given a data dictionary and tuple of x, y.
def get_series(df, x, y):
    return df[x], df[y]

# Get the title for two keys.
def get_matrix_title(x, y):
    return f"{get_title(x)} vs. {get_title(y)}"

# Plot scatterplots for each unique pairing of features.
def plot_matrices(df, figsize=(12,12), **kwargs):
    
    # Loop through each unique pair of keys.
    for (x, y) in get_unique_pairs(df):
                
        # Get matrix title.
        mat_title = get_matrix_title(x, y)
        
        # Get filename.
        mat_outfile = get_filename(mat_title, 'figures')
        
        # Get x label, y label.
        mat_xlab = get_label(x)
        mat_ylab = get_label(y)
        
        # Get the data.
        series_x, series_y = get_series(df, x, y)
        
        # Get x ticks, y ticks
        mat_xticks = get_ticks(np.min(series_x), np.max(series_x))
        mat_yticks = get_ticks(np.min(series_y), np.max(series_y))
                
        # Plot the matrix.
        plot_matrix(series_x, series_y, mat_title, mat_xlab, mat_ylab, mat_xticks, mat_yticks, mat_outfile, figsize=figsize, **kwargs)

In [29]:
%time plot_matrices(df_analysis, c='red', s=200, alpha=0.5)

CPU times: user 289 ms, sys: 9.64 ms, total: 298 ms
Wall time: 296 ms


> Now find the “Top 5” largest feature values for each of the features and note which data instances they belong to. For example, if the largest value for feature F1 is at index 2, then that would correspond to Instance 3. Fill out the table below by listing the instance numbers in sorted ascending order.

In [30]:
# 1. Given an array of values, map to (index, value).
# 2. Sort in descending order, so that the first item is the maximum value.
# 3. Discover the instance from the indices in the top 5 positions.
def sort_instances(series):
    # Enumerate instances.    
    instances = [ (index, value) for (index, value) in enumerate(series) ]
    # Sort in descending order and return.
    instances.sort(key=lambda x: x[1], reverse=True)
    return instances

In [31]:
# Repeat for each feature.
def sort_features(df):
    # Sort dictionary data per feature and return.
    return {
        key: sort_instances(values)
        for key, values
        in df.items()
    }

In [32]:
# Get the top 5 instances.
def find_top_series(series, n = 5):
    return sort_instances(series)[:n]

In [33]:
# Get the top 5 instances from each feature.
def find_top_instances(df, n = 5):
    for key, values in df.items():
        top_values = find_top_series(values, n=n)
        top_instances = [ f"[{instance}]={value:.2f}" for instance, value in top_values ]
        title = get_title(key)
        print(f"Top instances for {title} ({key}): {', '.join(top_instances)}")

In [34]:
# Get the top 5 for each feature.
find_top_instances(df_analysis, n = 5)

Top instances for Peak Linear Acceleration (PLA): [19]=151.71, [8]=131.70, [10]=130.73, [18]=127.11, [7]=110.94
Top instances for Average Rotational Velocity (ARV): [7]=23.40, [59]=21.51, [13]=21.36, [24]=20.68, [8]=20.29
Top instances for Peak Rotational Acceleration (PRA): [8]=44.54, [18]=30.29, [7]=27.93, [10]=25.11, [19]=23.07


> Two of the features should have the exact same instance numbers in the table above, which means that these are the five “different” data instances. Based on what you learned from our discussion of visual separability in class, think about how the fact that two features have the same five “different” data instances manifests itself in the scatter plot for those two features, as compared with the other two feature scatter plots.

The two 'different' features are likely correlated with one another. While it may not prove a causual relationship, the implications of association can mean a lot depending on the context of the analysis.